##### Checking if there are any features which don't update significantly during chemical evolution.

In [4]:
import os
import re
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from joblib import dump, load
import numpy as np
import pandas as pd
import optuna
import gc
import torch
from torch import nn
from joblib import load, dump
from datetime import datetime
from torch.nn import functional as F
import torch.distributed as dist
import os
from torch.distributed import init_process_group, destroy_process_group

model_timesteps = 90
fraction_of_dataset = 0.0001

def rename_columns(column_name):
    return column_name.replace('Plus', '+').replace('Num', '#').replace('At', '@')

testing_columns = []
useless_features = ["zeta", "Time", "Model"]
physical_features_columns = ['gasTemp', 'av', 'Radfield', 'Density']
data_filepath = '/data/uclchem_testing_data.h5'
num_features = 343 - len(physical_features_columns)

i = 751

dataset = pd.read_hdf(data_filepath, 'uclchem_models', start=i*model_timesteps, stop=(i+1)*model_timesteps).drop(columns=physical_features_columns).drop(columns=useless_features).drop(columns=testing_columns)
df_sorted_by_sum = dataset[dataset.sum().sort_values(ascending=False).index]
df_sorted_by_sum.rename(columns=lambda x: rename_columns(x), inplace=True)


# df_sorted_by_sum = df_sorted_by_sum.sort_index(axis=1)
# df_columns = df_sorted_by_sum.columns.tolist()

# pd.Series(df_columns).to_csv("/home/uclchem_emulator/utils/species.csv", index=False, header=False)
# pd.Series(physical_features_columns).to_csv("/home/uclchem_emulator/utils/physical_parameters.csv", index=False, header=False)


# species_columns = pd.read_csv("/home/uclchem_emulator/utils/species.csv", header=None).squeeze().tolist()
# physical_parameters_columns = pd.read_csv("/home/uclchem_emulator/utils/physical_parameters.csv", header=None).squeeze().tolist()

# print(species_columns)
# print(physical_parameters_columns)

In [5]:
atomic_masses = {
    'H': 1.0,
    'HE': 4.0,
    'C': 12.0,
    'N': 14.0,
    'O': 16.0,
    'S': 32.0,
    'SI': 28.0,
    'MG': 24.0,
    'CL': 35.5,
}

def calculate_abundance(df_row, pattern):
    total_abundance = 0
    for col in df_row.index:
        match = pattern.search(col)
        if match:
            multiplier = int(match.group(1)) if match.group(1) else 1
            total_abundance += df_row[col] * multiplier
    return total_abundance

def calculate_molecular_mass(species):
    if species in ["SURFACE", "BULK"]:
        return 0.0
    mass = 0.0
    speciesTemp = species
    
    pattern_multi = re.compile(r'(HE|SI|CL)(\d*)')
    for element, count in pattern_multi.findall(species):
        count = int(count) if count else 1
        mass += atomic_masses[element] * count
        speciesTemp = re.sub(f'{element}{count if count > 1 else ""}', '', speciesTemp)

    pattern_single = re.compile(r'(H|C|N|O|S|MG)(\d*)')
    for element, count in pattern_single.findall(speciesTemp):
        count = int(count) if count else 1
        mass += atomic_masses[element] * count

    return mass

def check_conservation(df_row):
    elements_patterns = {
        'H': re.compile(r'H(?!E)(\d*)'),
        'HE': re.compile(r'HE(\d*)'),
        'C': re.compile(r'C(?!L)(\d*)'),
        'N': re.compile(r'N(\d*)'),
        'O': re.compile(r'O(\d*)'),
        'S': re.compile(r'S(?!I)(\d*)'),
        'SI': re.compile(r'SI(\d*)'),
        'MG': re.compile(r'MG(\d*)'),
        'CL': re.compile(r'CL(\d*)')
    }
    
    abundances = {element: calculate_abundance(df_row, pattern) for element, pattern in elements_patterns.items()}
    
    total_mass = sum(df_row[col] * calculate_molecular_mass(col) for col in df_row.index)
    
    E_abundance = sum(df_row[col] for col in df_row.index if "+" in col)

    print(f"Mass={total_mass:.12e} | Electrons=({E_abundance:.12e}, {df_row['E_minus']:.12e}) | H={abundances['H']:.12e} | HE={abundances['HE']:.12e} | C={abundances['C']:.12e} | N={abundances['N']:.12e} | O={abundances['O']:.12e} | S={abundances['S']:.12e} | SI={abundances['SI']:.12e} | MG={abundances['MG']:.12e} | CL={abundances['CL']:.12e}")


i = 751
for i in range(750, 1000, 1):
    dataset = pd.read_hdf(data_filepath, 'uclchem_models', start=i*model_timesteps, stop=(i+1)*model_timesteps).drop(columns=physical_features_columns).drop(columns=useless_features).drop(columns=testing_columns)
    df_sorted_by_sum = dataset[dataset.sum().sort_values(ascending=False).index]
    df_sorted_by_sum.rename(columns=lambda x: rename_columns(x), inplace=True)
    for ind in range(2):
        df_row = df_sorted_by_sum.iloc[ind]
        check_conservation(df_row)
    print()

Mass=1.408550427441e+00 | Electrons=(1.917136749732e-04, 1.917129993672e-04) | H=9.999997008367e-01 | HE=1.000000034704e-01 | C=1.770004751846e-04 | N=6.180002295926e-05 | O=3.339999490876e-04 | S=3.510000288499e-06 | SI=1.780001956801e-06 | MG=2.256000016132e-06 | CL=3.389997534464e-08
Mass=1.408550432111e+00 | Electrons=(1.917197161033e-04, 1.917189947562e-04) | H=9.999997004887e-01 | HE=1.000000034704e-01 | C=1.770004930732e-04 | N=6.180003936661e-05 | O=3.340002349420e-04 | S=3.510000288500e-06 | SI=1.780001956881e-06 | MG=2.256000016132e-06 | CL=3.389997640504e-08

Mass=1.408551199601e+00 | Electrons=(1.933240775078e-04, 1.929690042743e-04) | H=1.000000431050e+00 | HE=1.000000144614e-01 | C=1.770010876454e-04 | N=6.180000178817e-05 | O=3.340000558689e-04 | S=3.510888141829e-06 | SI=1.780004638972e-06 | MG=2.255996290639e-06 | CL=3.390001435120e-08
Mass=1.408550671971e+00 | Electrons=(1.941494655307e-04, 1.937589986483e-04) | H=9.999999859106e-01 | HE=9.999999382984e-02 | C=1.77001

In [ ]:
#gastemp = [5, 5000], zeta = 2.3077, av = [0.00625, 6250], radfield = [0.001, 5000], density = [10, 10000000]